In [1]:
import numpy as np
import re
import snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import os
from datetime import datetime
from datetime import date
from datetime import timedelta
from sqlalchemy import create_engine

# Search Tweets

## Functions

In [2]:
#Functions to set up and write on SQL database
from sqlite3 import Error 

def sql_connection(dbname):
    try:
      conn = sqlite3.connect(dbname)
      return conn
    except Error:
      print(Error)


def table_insert(tablename, conn, rows, col_assign):
    
    #table name = name the table you will put in
    #conn = connection, get it by the function above
    #rows = data to put in
    #col_assign = sql needs the values to be assigned
    try:
        colnum=len(rows[0])
        cursorObj = conn.cursor()
        cursorObj.execute("CREATE TABLE "+tablename+col_assign)
        sqlite_insert_query_format = "INSERT INTO {0} VALUES {1};"
        query_ques='('+''.join(['?,']*(colnum-1)+['?'])+')'
        sqlite_insert_query=sqlite_insert_query_format.format(tablename,query_ques)
        cursorObj.executemany(sqlite_insert_query, rows)
        conn.commit()      
        print("Number of records after inserting rows:")
        cursor = cursorObj.execute('SELECT * from '+tablename+';')
        print(len(cursor.fetchall()))
        
    except sqlite3.OperationalError:
        ans=input("The table already exists. Do you want to update it? Y/N: ")
        if ans == 'Y':
            sql_stmt = "DROP TABLE "+str(tablename)
            conn.cursor().execute(sql_stmt)
            table_insert(tablename, conn, rows, col_assign)
        else: ('Insertion Cancelled')
        
def sql_assign_str(colnames,listoflists):
    width=len(listoflists[0])
    df=pd.DataFrame(listoflists)
    dtypelist=df.infer_objects().dtypes
    result='('
    for i in range(0,width):
        if dtypelist[i] == object: entry_type='text'
        elif str(dtypelist[i]) == 'int64': entry_type='real' 
                
        if i ==0:
            result=result+colnames[i]+' '+entry_type
        else:
            result=result+', '+colnames[i]+' '+entry_type
    result=result+')'
    return result

#frequently used variables
assignstr_tweets='(Tweeted_Date text, Content text, Username text, Replies real, RTs real, Likes real, QRTs real, Source text, Hashtags text)'
assignstr_users='(Username text, Displayname text, Created_Date text, Followers real, Friends real, Statuses real, Favorites real, Listeds text, Medias text)'
conn=sql_connection('TwitterProject.db')



In [169]:
#Snscrape search functions
def search_scrape(search_word,limit):
    tweets = [] #initiate arrays
    users = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_word).get_items()):
        if i>limit:
            break
        t = tweet #for simplicity

        
        #SQLite does not take iterable items to be saved in their system.
        #I need to manually parse them later if I want to access to the hashtags
        #empty lists for placeholders for the tweets without hashtags
        try:
            tweets.append([t.date.date().strftime('%Y-%m-%d'), t.content,t.user.username, t.replyCount,
                            t.retweetCount, t.likeCount, t.quoteCount, t.sourceLabel, t.hashtags.str.join('-')])
        
            users.append([t.user.username, t.user.displayname, t.user.created.date().strftime('%Y-%m-%d'),
                            t.user.followersCount, t.user.friendsCount, t.user.statusesCount, t.user.favouritesCount,
                            t.user.listedCount, t.user.mediaCount])
        except AttributeError:
            tweets.append([t.date.date().strftime('%Y-%m-%d'), t.content,t.user.username, t.replyCount,
                            t.retweetCount, t.likeCount, t.quoteCount, t.sourceLabel, ''])
        
            users.append([t.user.username, t.user.displayname, t.user.created.date().strftime('%Y-%m-%d'),
                            t.user.followersCount, t.user.friendsCount, t.user.statusesCount, t.user.favouritesCount,
                            t.user.listedCount, t.user.mediaCount])
    return tweets, users

#Snscrape Hashtag search functions
def Hashtag_scrape(search_word,limit):
    tweets = [] #initiate arrays
    users = []
    for i,tweet in enumerate(sntwitter.TwitterHashtagScraper((search_word)).get_items()):
        if i>limit:
            break
        t = tweet #for simplicity

        
        #SQLite does not take iterable items to be saved in their system.
        #I need to manually parse them later if I want to access to the hashtags
        #empty lists for placeholders for the tweets without hashtags
        try:
            tweets.append([t.date.date().strftime('%Y-%m-%d'), t.content, t.user.username, t.replyCount,
                            t.retweetCount, t.likeCount, t.quoteCount, t.sourceLabel, t.hashtags.str.join('-')])
        
            users.append([t.user.username, t.user.displayname, t.user.created.date().strftime('%Y-%m-%d'),
                            t.user.followersCount, t.user.friendsCount, t.user.statusesCount, t.user.favouritesCount,
                            t.user.listedCount, t.user.mediaCount])
        except AttributeError:
            tweets.append([t.date.date().strftime('%Y-%m-%d'), t.content, t.user.username, t.replyCount,
                            t.retweetCount, t.likeCount, t.quoteCount, t.sourceLabel, ''])
        
            users.append([t.user.username, t.user.displayname, t.user.created.date().strftime('%Y-%m-%d'),
                            t.user.followersCount, t.user.friendsCount, t.user.statusesCount, t.user.favouritesCount,
                            t.user.listedCount, t.user.mediaCount])
    return tweets, users

#Identify users, and get their second tweets (possibly not the tweet we searched for)
#Bad time complexity. Don't try too many
def Scraped_users_sixth(search_word,limit):
    tweets = [] #initiate arrays
    users = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper((search_word)).get_items()):
        if i>limit:
            break
        t = tweet #for simplicity
        if t.user.statusesCount>10: #If the user made more than 10 tweets (active):
            users.append([t.user.username, t.user.displayname, t.user.created.date().strftime('%Y-%m-%d'),
                            t.user.followersCount, t.user.friendsCount, t.user.statusesCount, t.user.favouritesCount,
                            t.user.listedCount, t.user.mediaCount])
            
            t2=search_scrape('from:@'+t.user.username,20)[0][5]
            tweets.append(t2)                
        else: pass
    return tweets, users   
            

I found that my functions didn't handle hashtag collection error well and made everything empty string. I will get it back in the data anlaysis stage with parsing

## Clean search
Seraching for tweets that's ostensibly Clean. Will analyze and prove how clean the search are in the analysis stage.
Clean means tweets and accounts made by real users, not spams accounts. It may include people who are using Twitter as vulgar uses, but I don't defie them as 'dirty' as they are real users and do not produce illicit promotional posts

### Tweets about Trending Keywords

In [254]:
import urllib
from urllib.parse import quote_plus
from urllib.parse import unquote_plus

scraper = sntwitter.TwitterTrendsScraper()

keywords=[]
for trend in scraper.get_items():
    print(unquote_plus(str(trend)))
    keywords.append(str(trend).partition('?q=')[2])

#twitter provides 20 trending keywords by default


https://twitter.com/search?q=바퀴벌레
https://twitter.com/search?q=#에스엠_레드벨벳_콘서트_돌려놔
https://twitter.com/search?q=임창균 우비
https://twitter.com/search?q=소속사 이적
https://twitter.com/search?q=서울페스타
https://twitter.com/search?q=괜찮으신가요
https://twitter.com/search?q=무정부상태
https://twitter.com/search?q=전석 13만원
https://twitter.com/search?q=콘서트 취소
https://twitter.com/search?q=인스타 압수
https://twitter.com/search?q=양성애자
https://twitter.com/search?q=인명피해
https://twitter.com/search?q=워터파크
https://twitter.com/search?q=인스타 해킹
https://twitter.com/search?q=잠실 시야
https://twitter.com/search?q=헌트 후기
https://twitter.com/search?q=주경기장
https://twitter.com/search?q=남부지방
https://twitter.com/search?q=집중호우
https://twitter.com/search?q=재택근무


In [ ]:
#List of people speaking about trending posts

key_tweets = []
key_users = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for keyword in keywords:
    kt,ku = search_scrape(keyword,100) #20 keywords, 
    key_tweets.append(kt)
    key_users.append(ku)

#hierarchy key_tweets/users[index of keyword][index of each tweet][index of items]
#use concat to make it a flattened list, but we sometimes have to kick out some automated tweet's keywords
#manually check it out

In [277]:
pd.DataFrame(key_tweets).to_csv('key_tweets.csv')
pd.DataFrame(key_users).to_csv('key_users.csv')

In [274]:
#print search results and see the quality of tweets

for i in range(0,20):
    print(i)
    print(key_tweets[i][5])
    print()

0
['2022-08-10', '너 강남사는 바퀴벌레지 https://t.co/ttwqZHQkFg', 'latte_0425', 0, 0, 0, 0, 'Twitter for Android', '']

1
['2022-08-10', '@RVsmtown BETTER TREATMENT FOR @RVsmtown \n\n#SM_여돌차별_공론화\n#에스엠_레드벨벳_콘서트_돌려놔 \n@SMTOWNGLOBAL', 'moaluvieeeee', 0, 0, 0, 0, 'Twitter for Android', '']

2
['2022-08-10', '임창균 직캠\n임창균 셀카\n임창균 금발\n임창균 흑발\n임창균 랩 찢어라\n임창균 우비\n임창균 갓댐\n임창균 막내\n임창균 고양이\n임창균 귀여워\n임창균 사랑해 https://t.co/wCHzPyZgPk', 'kyunrill', 0, 0, 0, 0, 'Twitter for iPad', '']

3
['2022-08-10', '하성운, 24일 미니 7집 발매 확정…소속사 이적 후 첫 앨범[공식] (출처 : 뉴스엔 | 네이버 TV연예) https://t.co/aHm9ieKc2P', 'sarangsungwoon', 0, 0, 0, 0, 'Twitter for Android', '']

4
['2022-08-10', '서울페스타 엔시티드림 팬석 3층 35구역 2열 양도합니다 (34,36 사이에 구역 있습니다)\n\n1장 3.0\n\n팔찌랑 티켓 다 드립니다 공연장 앞 직거래 가능\n\n엔시티드림 서울페스타 시즈니 팬석 믐뭔봄 제노 해찬 마크 지성 천러 런쥔 재민 NCT NCT DREAM nctdream', 'dhhwsyyshwhzjjw', 0, 0, 0, 0, 'Twitter for iPhone', '']

5
['2022-08-10', '@minggu_art 콘서트 중간에 쓰러지셔서 의무실러 데랴왔습니다ㅠㅠ 괜찮으신가요?!', 'Ol_Moonlight', 0, 0, 0, 0, 'Twitter for Android', '']

6
['20

In [271]:
#flatten the list to hierarcy: key_tweets/users[index of each tweet][index of items]
key_tweets_flat=[]
key_users_flat=[]
for i in range(0,20):
    key_tweets_flat=key_tweets_flat+key_tweets[i]
    key_users_flat=key_users_flat+key_users[i]

### Tweets about Official Twitter Creators

In [174]:
#Who are either twitter official creators or talking about official creators
#This is an amazingly clean keyword
#I assume twitter is keeping their eyes on this query for the quality of their promotions on creators
Creator_Tweets, Creator_Users = search_scrape('트위터 크리에이터',500)

['2022-08-08', '‘개발자가 되고 싶다면 트위터하세요’ \n트위터 크리에이터 프로그램에 참여하신 클로이님 @chloeelog 인터뷰가 매경에 실렸습니다.\n트위터만의 독특한 확장성과 크리에이터 생태계가 잘 드러난 기사 감사드려요👍@HyejiPark5 @damdadi_dami \nhttps://t.co/BnGDW5THUD', 'jangp0p', 0, 4, 8, 0, 'Twitter for iPhone', ''] ['jangp0p', 'Grace Jang', '2011-09-29', 148, 795, 593, 894, 9, 96]


In [182]:
#One blatant spam tweet found
print(Creator_Tweets[2])

['2022-08-07', '▼▼ㅇㅑ밤공식ㅌㅡ위ㅌㅓ▲♠▽\n\n#FLO_크리에이터_모집 #망가ㅅㅏ이트\n#ㅅㅡㅌㅠㅇㅓㄷㅣㅅㅡ과외\n\n♠주_소  https://t.co/G0rvSL3MJQ\n\n늘봄넷긴급 https://t.co/vx4CFWe8TA', 'loveegg_0408', 0, 0, 0, 0, 'Twitter Web App', '']


In [183]:
#pop it out
Creator_Tweets.pop(2)
Creator_Users.pop(2)

len(Creator_Tweets)

500

### Idol Foodie Hashtags

In [119]:
idol_hashtags=('위EAT_인EAT','승윤이의_맛집_PAGE','이맛집_찐받네','마이노_마이무','훈슐랭_여기도가봤니','정한아_마니머거여ㅎㅎ','조슈아_먹어보슈아','순영아_이거_맛있어'
,'원우야_여기_테이스티','도겸이도_도아할_맛집','박지성_단1g도안줌','제노의_맛그당어','나나의_맛집탐방기','정우의_원데이씩스밀','먹어봤도영','백현이를_위한_맛집투어'
,'몬베베가_몬베베에게_추천하는_맛집','채형원_너는세입도못먹겠지','지민이와_밥밥밥을먹어요','정국아_같이먹짱','머거스트디','뷔슐랭','기범아_나혼자먹는거_아니야'
,'ㄷㅂㅇㅈ','이션같이찌자','오늘은웬디여기','강다니엘_염염긋','우석아_맛집으로_우따따따','승연아_우즈야_여기야','황제님을위한메뉴판','예의있게_추천해'
,'지훈아_여기가_맛있지훈','김재환_밥은_잘먹고있냐')

#Got these hashtags from https://twitter-michelin-guide.netlify.app/hashtag.html
#Most searched hashtags used by Kpop idol fandoms to suggest their restraunt recommendations to their fans and idols
#Idol fans keep the quality of their hashtags clean so that their idols can be exposed more with searches and retweets

In [132]:
Hash_Tweets=[]
Hash_Users=[]
for tag in idol_hashtags:
    ht, hu= Hashtag_scrape(tag,50)
    Hash_Tweets=Hash_Tweets+ht
    Hash_Users=Hash_Users+hu
print(len(Hash_Tweets))

1657


### Who opened Spaces, limited to Korean Tweets

In [184]:
#Users who recently opened a space, and their 6th recent tweets
#Space is a public voicechat of which their 'open' action is given to active users for AB testing 
Space_Tweets,Space_Users=Scraped_users_sixth('filter:spaces lang:ko',1000)
print(len(Space_Tweets))
print(Space_Tweets[2])

1001
['2022-08-08', '@ilove_lve 헐 깅 멋쪄요..😍', 'ihaveive123', 1, 0, 0, 0, 'Twitter for Android', '']


## To SQL database

In [187]:
#Data types of tweets and users are same for all results, so made a universal SQL assignment string
tweet_assignstr=sql_assign_str(['Tweeted_Date','Content','Username','Replies','RTs','Likes','QRTs','Source','Hashtags'],key_tweets_flat)
user_assignstr=sql_assign_str(['Username','Displayname','Created_Date','Followers','Friends','Statuses','Favorites','Listeds','Medias'],key_users_flat)

#2020 items, Based on Trending Keywords(Aug-10-2022)
table_insert('Key_Tweets', conn, key_tweets_flat, tweet_assignstr)
table_insert('Key_Users', conn, key_users_flat, user_assignstr)

#500 items, Search Result of "트위터 크리에이터", One spam tweet manually identified and dropped
table_insert('Creator_Tweets', conn, Creator_Tweets, tweet_assignstr)
table_insert('Creator_Users', conn, Creator_Users, user_assignstr)

#1657 items, Search Result of Poplular idol foodie hashtags
table_insert('Hash_Tweets', conn, Hash_Tweets, tweet_assignstr)
table_insert('Hash_Users', conn, Hash_Users, user_assignstr)

#1001 items, Users who recently opened a space (public voicechat), and their 6th recent tweets
table_insert('Space_Tweets', conn, Space_Tweets, tweet_assignstr)
table_insert('Space_Users', conn, Space_Users, user_assignstr)



The table already exists. Do you want to update it? Y/N: Y
Number of records after inserting rows:
1818
The table already exists. Do you want to update it? Y/N: Y
Number of records after inserting rows:
1818
The table already exists. Do you want to update it? Y/N: Y
Number of records after inserting rows:
500
The table already exists. Do you want to update it? Y/N: Y
Number of records after inserting rows:
500
Number of records after inserting rows:
1657
Number of records after inserting rows:
1657
Number of records after inserting rows:
1001
Number of records after inserting rows:
1001


## Dirty Search

### Manually Found Targeted Keyword

In [202]:
#Targeted Keyword recognition -> Classify it into general and specific
JJ_Tweets, JJ_Users = search_scrape('제주아이스크림 since:2022-05-01',300)


### Communication Methods Used by Spam Accounts

In [204]:
#display name includes :ㄹㅏ인 ㄹr인...
Line_Tweets = []
Line_Users = []
lt,lu = search_scrape('ㄹr인',500)
Line_Tweets = Line_Tweets + lt
Line_Users = Line_Users +lu
lt,lu = search_scrape('ㄹㅏ인',500)
Line_Tweets = Line_Tweets + lt
Line_Users = Line_Users +lu

## To SQL database

In [206]:
#301 items, specific targeted Keyword I manually found
table_insert('Jeju_Tweets', conn, JJ_Tweets, tweet_assignstr)
table_insert('Jeju_Users', conn, JJ_Users, user_assignstr)

#1002 items, Usernames including contact platform often used for illicit convos
table_insert('Line_Tweets', conn, Line_Tweets, tweet_assignstr)
table_insert('Line_Users', conn, Line_Users, user_assignstr)



Number of records after inserting rows:
110
Number of records after inserting rows:
110
Number of records after inserting rows:
1002
Number of records after inserting rows:
1002


## Save SQL database into csv
I'm unfamiliar with SQL on python yet, so I want to save data in csv just in case I didn't safely saved data in TwitterProject.db

In [278]:
for item in ('Jeju','Line','Hash','Space','Key','Creator'):
    df=pd.read_sql('''SELECT DISTINCT *
                        FROM {0} 
                        LEFT JOIN {1} 
                        ON {0}.Username = {1}.Username
                        '''.format(item+'_Users',item+'_Tweets'),conn
                        )
    df.to_csv(item+'_join.csv')
    print(df.Content.head(10))


0                      ⌚\n지글지글 사하구맛집 린헤어 제주아이스크림맛집 \n📓
1                            🐼\n제주아이스크림 여행 일탈 #호기심 \n📿
2                      💞\n지글지글 사하구맛집 린헤어 제주아이스크림맛집 \n🐍
3                           🍉\n제주아이스크림맛집 인턴 알바 #혼술 \n⏸
4                       ‍♂️\n제주아이스크림맛집 인턴 알바 #혼술  \n🐈‍
5                               ♾\n제주아이스크림맛집 인턴 알바 #혼술
6                               🍣\n제주아이스크림맛집 인턴 알바 #혼술
7    타이밍\n책상인테리어\n천일\n천안호두\n힘이되는글\n떠나고싶다\n상서구룸싸룽\n상...
8    제주시카페\n송도맛집마포소금구이\n태백코로나\n무역\n셀프레벨링업체\n촬영소품\n서...
9    패스트픽\n브라운브레스\n남원다방\n리조또\n양산시모텔\n박자\n채식식단\n시청후\...
Name: Content, dtype: object
0          은평출장안ㅁㅏ\n출장호텔\nW2Ena\n은평유부녀\n#섹트
1       ‍❤️‍\n정읍출장만남 아가씨  천안출장안마  셀기꾼  \n❤️
2       ‍🔥\n강원테라피출장  선팔환영 밀양  하남동출장마사지  \n🧡
3    ❣\n사천탕수육  전공원 애니메이트강남역채팅후기  진주개인스냅 \n💞
4          ❤️‍\n배우  사천페이만남 경기악화  유산소운동  \n💖
5                💓\n오겹살 오늘의유머  오프남 일상소통 \n🧡
6              💕\n보성맛집 롱다리  육변기 사상출장안마  \n🧡
7              💕\n빙수 키스방   비토성감 그로우커넥트  \n❣
8           💕\n수원만남  포항당일만남사이트  코디  횡성군 \n💓
9               💕\n출장마사지제